<a href="https://colab.research.google.com/github/sherkhalil/COVID19/blob/master/BERT_Classifier_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dataset = '/content/drive/My Drive/Jan9-2012-tweets-clean (version 1).csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
data = pd.read_csv(dataset) 
data.head()

,Id,Tweet,Label
0,145353048817012000,Thinks that @melbahughes had a great 50th birt...,surprise
1,144279638024257000,"Como una expresiÃ³n tan simple, una sola oraci...",sadness
2,140499585285111000,the moment when you get another follower and y...,joy
3,145207578270507000,Be the greatest dancer of your life! practice ...,joy
4,139502146390470000,eww.. my moms starting to make her annual rum ...,disgust


In [ ]:
data.Label.value_counts()

joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: Label, dtype: int64

In [ ]:
data.drop(data.loc[data.Label == 'joy'].index, inplace=True)
data.drop(data.loc[data.Label == 'surprise'].index, inplace=True)

#data.drop(data.loc[data.Label == 'sadness'].index, inplace=True)
#data.drop(data.loc[data.Label == 'fear'].index, inplace=True)
#data.drop(data.loc[data.Label == 'anger'].index, inplace=True)
data.drop(data.loc[data.Label == 'disgust'].index, inplace=True)
data.Label.value_counts()

sadness    3830
fear       2816
anger      1555
Name: Label, dtype: int64

In [ ]:
data['Label_Numeric'] = data['Label'].map({'sadness':0, 'fear':1,'anger':2})

In [ ]:
data.Label_Numeric.value_counts()

0    3830
1    2816
2    1555
Name: Label_Numeric, dtype: int64

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 1.8MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30317 sha256=9a69642fd315f7af538bb3cae71a90729358eab6dd1ec24239bdc5462c8b4277
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=ebad41b7f76f05cd2019191df80f68a2cb1bbaa65a4ad93efa35566d46c93dad
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=9d9f68ebdd4d839cf1a7d9bf8635b4291dad590b9991e350711a69784235f81c
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 2.8MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
data.shape

(8201, 4)

In [ ]:
data['Tweet']

1        Como una expresiÃ³n tan simple, una sola oraci...
8        Making art and viewing art are different at th...
9        Soooo dooowwwn!! Move on, get some sleep... Me...
10       &quot;We're sorry, but the clip you selected i...
13       My heart and soul @Jay_BabeBee is leaving me a...
                               ...                        
21037    Even though I walk through the valley of the s...
21042    Outfit della digital hypster: tavola da snowbo...
21043    Have to learn for history... But I can't conce...
21045    About to have a movie night with my booboo @je...
21050    &quot;Use to throw them into disarray.&quot;-M...
Name: Tweet, Length: 8201, dtype: object

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
reviews = []
sentences = list(data['Tweet'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [ ]:
print(data.columns.values)

['Id' 'Tweet' 'Label' 'Label_Numeric']


In [ ]:
data.Label_Numeric.unique()

array([0, 1, 2])

In [ ]:
import numpy as np
y = data['Label_Numeric']

y = np.array(list(map(lambda x: 1 if x==1 else (2 if x==2 else 0), y)))

In [ ]:
y

array([0, 1, 2, ..., 1, 0, 2])

In [ ]:
reviews[10]

'My earlier tweet should ve said quot sometimes think m the youngest person quot '

In [ ]:
y[10]

2

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
tokenized_reviews[0]

[18609,
 14477,
 4654,
 28994,
 2072,
 9092,
 3722,
 14477,
 14017,
 2050,
 2030,
 6305,
 2072,
 16405,
 14728,
 2222,
 29107,
 2527,
 4830,
 16185,
 9092,
 3406]

In [ ]:
reviews_with_len = [[review, y[i], len(review)]  for i, review in enumerate(tokenized_reviews)]

In [ ]:
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 4), dtype=int32, numpy=
 array([[ 2146,  4632,  2154,     0],
        [ 3402,  2514,  8211,     0],
        [ 2155,  4596, 26256,     0],
        [ 5091,  4308, 20398,     0],
        [ 3985,  3360,  4619,     0],
        [ 2082,  2651,  6682,     0],
        [ 3398,  3835, 18975,     0],
        [ 3110,  4064,  2503,     0],
        [ 2968,  6832, 22422,     0],
        [ 2697,  3466,  4586,     0],
        [ 3147, 11771,  2305,     0],
        [ 2975,  3409,  2651,     0],
        [10166,  2919,  3944,     0],
        [ 5782,  2980,  7967,     0],
        [12809,  3147,  6457,     0],
        [ 3241, 27824, 11891,     0],
        [ 3428,  2267,  7665,     0],
        [ 5236,  5236,  5236,     0],
        [ 4689,  5236,  2293,     0],
        [ 4485,  4234,  6023,     0],
        [ 5236, 16522,  2338,     0],
        [ 3288,  2067,  5387,     0],
        [14743,  2000,  2681,  4826],
        [ 9975,  4468, 17115,  4784],
        [ 2053,  3788,  2757,  3892],
  

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="BERT_Classifier_C"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 3

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
232/232 [==============================] - 19s 83ms/step - loss: 0.8849 - sparse_categorical_accuracy: 0.5978
Epoch 2/5
232/232 [==============================] - 19s 82ms/step - loss: 0.4464 - sparse_categorical_accuracy: 0.8256
Epoch 3/5
232/232 [==============================] - 19s 82ms/step - loss: 0.1134 - sparse_categorical_accuracy: 0.9662
Epoch 4/5
232/232 [==============================] - 19s 81ms/step - loss: 0.0221 - sparse_categorical_accuracy: 0.9964
Epoch 5/5
232/232 [==============================] - 19s 82ms/step - loss: 0.0092 - sparse_categorical_accuracy: 0.9985


In [ ]:
results = text_model.evaluate(test_data)
print(results)

25/25 [==============================] - 0s 9ms/step - loss: 1.5148 - sparse_categorical_accuracy: 0.6488
[1.5148227214813232, 0.6487500071525574]


In [ ]:
text_model.summary()


Model: "BERT_Classifier_C"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6104400   
_________________________________________________________________
conv1d (Conv1D)              multiple                  40100     
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  77056     
_________________________________________________________________
dropout (Dropout)            multiple            